In [1]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

4.45.2
C:\Users\T-Plug\anaconda3\envs\social_media_mining_env\Lib\site-packages\transformers\__init__.py


In [2]:
import torch
print(torch.__version__)
print(torch.__file__)


2.8.0+cpu
C:\Users\T-Plug\anaconda3\envs\social_media_mining_env\Lib\site-packages\torch\__init__.py


In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, set_seed
from sklearn.metrics import accuracy_score, f1_score

# Load data
train = pd.read_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined/combined_train.csv")
dev = pd.read_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined/combined_dev.csv")
test = pd.read_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/Datasets/Combined/combined_test.csv")

# Map labels to integers
label_map = {label: i for i, label in enumerate(train['label'].unique())}
train['label'] = train['label'].map(label_map)
dev['label'] = dev['label'].map(label_map)
test['label'] = test['label'].map(label_map)

train_ds = Dataset.from_pandas(train)
dev_ds = Dataset.from_pandas(dev)
test_ds = Dataset.from_pandas(test)

# Tokenizer + model
model_name = "castorini/afriberta_base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize_fn(batch):
    return tokenizer(batch['tweet'], truncation=True, padding="longest", max_length=64)

train_ds = train_ds.map(tokenize_fn, batched=True)
dev_ds = dev_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_map))

# Training setup
set_seed(42)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "macro_f1": f1_score(labels, preds, average="macro")
    }

training_args = TrainingArguments(
    output_dir="./results/afriberta_exp1",
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    compute_metrics=compute_metrics,
)

print("Setup complete. You can now run trainer.train() in a separate cell.")


C:\Users\T-Plug\anaconda3\envs\social_media_mining_env\Lib\site-packages\transformers\convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/7620 [00:00<?, ? examples/s]

Map:   0%|          | 0/1275 [00:00<?, ? examples/s]

Map:   0%|          | 0/1837 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at castorini/afriberta_base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setup complete. You can now run trainer.train() in a separate cell.


In [2]:
# trainer_small.train()
trainer.train()


C:\Users\T-Plug\anaconda3\envs\social_media_mining_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
0,0.832600,0.728667,0.701961,0.616942


C:\Users\T-Plug\anaconda3\envs\social_media_mining_env\Lib\site-packages\transformers\configuration_utils.py:384: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512}
  warnings.warn(
C:\Users\T-Plug\anaconda3\envs\social_media_mining_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

In [ ]:
# !pip install --upgrade "accelerate>=0.26.0"

In [ ]:
train_results = trainer.evaluate(eval_dataset=train_ds)
print("\n Train Set Results:")
print(train_results)

dev_results = trainer.evaluate(eval_dataset=dev_ds)
print("\n Dev Set Results:")
print(dev_results)

test_results = trainer.evaluate(eval_dataset=test_ds)
print("\n Test Set Results:")
print(test_results)
